# Lesson 5: Debugging

In this lesson, you'll learn debugging techniques - how to find and fix bugs in your code.

---

## How to Start These Lessons

If you're reading this in a text editor and need to open it in Jupyter:

```bash
cd /path/to/pyside_tutorial
python start_lessons.py
```

**To stop the Jupyter server:** Press `Ctrl+C` in the terminal.

---

## What You'll Learn

- How to use print statements effectively
- Using debug flags to turn debugging on and off
- Reading error messages
- Common debugging strategies
- Why we'll eventually want something better (tests!)

---

## The Reality of Programming

Here's a secret: **professional programmers spend more time debugging than writing new code.**

Bugs are normal. Every program has them. The skill isn't avoiding bugs entirely - it's finding and fixing them quickly.

In this lesson, we'll learn the most basic debugging technique: **print statement debugging**.

---

## Print Statement Debugging

The simplest debugging technique: add `print()` statements to see what's happening inside your code.

In [ ]:
# A function with a bug
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total = num  # Bug! Should be: total += num
    return total / len(numbers)

result = calculate_average([10, 20, 30])
print(f"Average: {result}")  # Expected: 20, Got: 10

The result is wrong. Let's add print statements to find the bug:

In [ ]:
# Same function with debug prints
def calculate_average(numbers):
    print(f"DEBUG: Input numbers = {numbers}")
    total = 0
    for num in numbers:
        print(f"DEBUG: Processing {num}, total before = {total}")
        total = num  # Bug!
        print(f"DEBUG: total after = {total}")
    print(f"DEBUG: Final total = {total}, count = {len(numbers)}")
    return total / len(numbers)

result = calculate_average([10, 20, 30])

Now we can see the bug! The total should be accumulating (10, 30, 60) but it's being replaced each time (10, 20, 30).

The fix: change `total = num` to `total += num`

In [ ]:
# Fixed function
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num  # Fixed!
    return total / len(numbers)

result = calculate_average([10, 20, 30])
print(f"Average: {result}")  # Now correctly shows 20.0

---

## The Debug Flag Pattern

The problem with print debugging: you have to add and remove print statements constantly.

A better approach: use a **debug flag** to turn debugging on and off.

Here's how it works in our actual project (`media_catalogue/src/media_catalogue.py` line 1-2):

In [ ]:
# Toggle on and off to see debug output
debug = True

class Movie:
    def __init__(self, title, year, director, duration):
        if debug:
            print(f"DEBUG: Creating Movie '{title}'")
        
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
        
        if debug:
            print(f"DEBUG: Movie created successfully")
    
    def __str__(self):
        return f"{self.title} ({self.year})"

# With debug = True, we see the messages
movie = Movie("The Matrix", 1999, "Wachowskis", 136)

In [ ]:
# Change debug to False - no more messages!
debug = False

class Movie:
    def __init__(self, title, year, director, duration):
        if debug:
            print(f"DEBUG: Creating Movie '{title}'")
        
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
        
        if debug:
            print(f"DEBUG: Movie created successfully")
    
    def __str__(self):
        return f"{self.title} ({self.year})"

# With debug = False, no messages
movie = Movie("Inception", 2010, "Nolan", 148)
print(movie)  # Only see the normal output

**Benefits of the debug flag:**
- Leave your debug prints in the code
- Toggle them on/off with one variable
- No need to add/remove prints constantly

---

## The Actual Debug Code in Our Project

Look at the bottom of `media_catalogue/src/media_catalogue.py` (lines 109-127):

In [ ]:
# From the actual project
debug = True

class MediaError(Exception):
    def __init__(self, message, obj):
        super().__init__(message)
        self.obj = obj

class Movie:
    def __init__(self, title, year, director, duration):
        self.title = title
        self.year = year
        self.director = director
        self.duration = duration
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.duration} min, {self.director}"

class TVSeries(Movie):
    def __init__(self, title, year, director, duration, seasons, total_episodes):
        super().__init__(title, year, director, duration)
        self.seasons = seasons
        self.total_episodes = total_episodes
    
    def __str__(self):
        return f"{self.title} ({self.year}) - {self.seasons} seasons, {self.total_episodes} episodes"

class MediaCatalogue:
    def __init__(self):
        self.items = []
    
    def add(self, item):
        self.items.append(item)
        return item
    
    def __str__(self):
        if not self.items:
            return "Media Catalogue (empty)"
        return f"Media Catalogue ({len(self.items)} items)"

# This code only runs when debug is True AND you run the file directly
if __name__ == "__main__" and debug == True:
    catalogue = MediaCatalogue()
    try:
        movie1 = Movie('The Matrix', 1999, 'The Wachowskis', 136)
        catalogue.add(movie1)
        movie2 = Movie('Inception', 2010, 'Christopher Nolan', 148)
        catalogue.add(movie2)

        series1 = TVSeries('Scrubs', 2001, 'Bill Lawrence', 24, 9, 182)
        catalogue.add(series1)
        series2 = TVSeries('Breaking Bad', 2008, 'Vince Gilligan', 47, 5, 62)
        catalogue.add(series2)
        
        print(f"\n{catalogue}")
        for item in catalogue.items:
            print(f"  - {item}")
    except ValueError as e:
        print(f'Validation Error: {e}')
    except MediaError as e:
        print(f'Media Error: {e}')
        print(f'Unable to add {e.obj}: {type(e.obj)}')

### Understanding `if __name__ == "__main__"`

This is a Python pattern:
- `__name__` is a special variable
- When you run a file directly, `__name__` is `"__main__"`
- When you import the file, `__name__` is the module name

So this code only runs when you execute the file directly, not when you import it.

---

## Reading Error Messages

Python error messages are actually helpful! Let's learn to read them:

In [ ]:
# This will cause an error
def broken_function():
    x = 10
    y = 0
    result = x / y  # Division by zero!
    return result

broken_function()

**Reading the error:**

1. **Traceback** - Shows the path through your code that led to the error
2. **File and line number** - Where the error occurred
3. **The actual code** - The line that caused the error
4. **Error type** - `ZeroDivisionError` tells you what went wrong
5. **Error message** - "division by zero" explains the problem

---

## Common Debugging Strategies

### 1. Binary Search Debugging

If you have a lot of code, add a print in the middle. If the bug is before the print, search the first half. If after, search the second half.

### 2. Check Your Assumptions

Print the values of variables you think you know:

In [ ]:
# You might assume numbers is a list of integers...
numbers = "123"  # But it's actually a string!

print(f"Type of numbers: {type(numbers)}")
print(f"Value of numbers: {numbers}")

### 3. Rubber Duck Debugging

Explain your code line-by-line to something (a rubber duck, a friend, yourself). Often you'll find the bug just by explaining it.

### 4. Take a Break

Seriously. Step away. Come back with fresh eyes. Many bugs are found this way.

---

## The Problem with Print Debugging

Print debugging works, but it has problems:

1. **Messy** - Print statements everywhere clutter your code
2. **Manual** - You have to run the code and read the output every time
3. **Not repeatable** - You can't easily re-run the same checks
4. **Not comprehensive** - You might miss edge cases
5. **Gets removed** - When you're done, you delete the prints and lose that work

**The solution? Tests!**

In the next lessons, we'll learn how to write tests that:
- Check your code automatically
- Run the same checks every time
- Catch bugs before they become problems
- Stay in your codebase forever

---

## Try It Yourself

### Exercise 1: Find the Bug

This function should return the sum of all even numbers, but it has a bug. Use print debugging to find it:

In [ ]:
# Exercise 1: Find the bug
def sum_evens(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:  # Check if even
            total += 1  # Bug! Should add num, not 1
    return total

result = sum_evens([1, 2, 3, 4, 5, 6])
print(f"Sum of evens: {result}")  # Expected: 12 (2+4+6), Got: 3

# Add print statements to find the bug, then fix it

<details>
<summary><b>Show Answer</b></summary>

The bug is on line 6: `total += 1` should be `total += num`

```python
def sum_evens(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            total += num  # Fixed!
    return total
```

</details>

### Exercise 2: Add a Debug Flag

Add a debug flag to this class so you can toggle debug output:

In [ ]:
# Exercise 2: Add debug flag
# Add a debug variable and use it to control print statements

class Counter:
    def __init__(self):
        self.count = 0
    
    def increment(self):
        self.count += 1
    
    def get_count(self):
        return self.count

counter = Counter()
counter.increment()
counter.increment()
print(f"Count: {counter.get_count()}")

<details>
<summary><b>Show Answer</b></summary>

```python
debug = True

class Counter:
    def __init__(self):
        if debug:
            print("DEBUG: Creating new counter")
        self.count = 0
    
    def increment(self):
        if debug:
            print(f"DEBUG: Incrementing from {self.count} to {self.count + 1}")
        self.count += 1
    
    def get_count(self):
        return self.count
```

</details>

---

## Key Takeaways

1. **Print debugging** is simple and effective - add `print()` to see what's happening

2. **Debug flags** let you toggle debug output without removing code

3. **Read error messages** - they tell you what went wrong and where

4. **Check your assumptions** - print variable types and values

5. **Print debugging has limits** - it's manual, messy, and not repeatable

6. **Tests are better** - they're automated, repeatable, and stay in your code

---

## Next Lesson

You'll learn:
- Why testing is better than print debugging
- How to write tests with pytest
- Writing your first test for the Movie class
- Running tests and reading results

---

## Navigation

| | |
|:---|---:|
| [**← Previous: Lesson 4 - Collections**](04_collections.ipynb) | [**Next: Lesson 6 - Testing Basics →**](06_testing_basics.ipynb) |

**Note:** If the link doesn't work (some browsers block popups), you can navigate manually:
1. Click the **Jupyter logo** in the top left to go to the file browser
2. Click on the next lesson file (e.g., `06_testing_basics.ipynb`)

Or use **File → Open** from the menu.